In [ ]:
# ChatGPT-4
import numpy as np
import matplotlib.pyplot as plt

# 定义f(x)


def f(x):
    return np.pi**2 * np.sin(np.pi * x)


# 网格划分
N = 100
h = 1.0 / N
x = np.linspace(0, 1, N+1)

# 基函数及其导数


def basis(i, xi):
    if i == 0:
        return (xi - h) / h
    elif i == N:
        return (h - xi) / h
    else:
        if xi <= x[i]:
            return (xi - x[i-1]) / h
        else:
            return (x[i+1] - xi) / h


def dbasis(i, xi):
    if i == 0 or i == N:
        return -1.0 / h
    else:
        if xi <= x[i]:
            return 1.0 / h
        else:
            return -1.0 / h


# 组装矩阵和向量
A = np.zeros((N+1, N+1))
b = np.zeros(N+1)

for i in range(1, N):
    for j in range(1, N):
        A[i, j] = np.sum([dbasis(i, xi)*dbasis(j, xi) for xi in x])*h
    b[i] = np.sum([f(xi)*basis(i, xi) for xi in x])*h

# 应用边界条件
# A = A[1:-1, 1:-1]
A[0, :] = 0
A[:, 0] = 0
A[N, :] = 0
A[:, N] = 0
A[0, 0] = 1
A[N, N] = 1
# b = b[1:-1]
b[0] = 1
b[N] = 1

A = A[1:-1, 1:-1]
b = b[1:-1]

# 求解线性系统
u = np.linalg.solve(A, b)

# 绘制结果
plt.plot(x[1:-1], u, 'o-', label='Numerical')
plt.plot(x, np.sin(np.pi*x), 'r--', label='Exact')
plt.legend()
plt.show()

In [ ]:
# ChatGPT-3.5
import numpy as np
from scipy.integrate import quad
from scipy.optimize import minimize

# 定义泊松方程的右侧函数 f(x)


def f(x):
    return -1.0

# 定义一维有限元法的基函数


def phi(i, x):
    if i == 0:
        return 0.5 - x
    elif i == 1:
        return 0.5 + x
    else:
        return 0.0

# 定义最小二乘目标函数
def objective(u_coeff):
    def u(x):
        return sum(u_coeff[i] * phi(i, x) for i in range(len(u_coeff)))

    def integrand(x):
        return (u(x) - f(x))**2

    result, _ = quad(integrand, 0, 1)
    return result


# 定义约束条件
constraints = ({'type': 'eq', 'fun': lambda u_coeff: u_coeff[0]},
               {'type': 'eq', 'fun': lambda u_coeff: u_coeff[-1]})

# 初始化系数
u_coeff_initial_guess = np.zeros(2)

# 最小化目标函数
result = minimize(objective, u_coeff_initial_guess, constraints=constraints)

# 提取最优系数
u_coeff_optimal = result.x

# 打印最优系数
print("Optimal u_coeff:", u_coeff_optimal)

In [ ]:
# Github Copilot
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

# 定义区间和步长
a, b = 0, 1
N = 100
h = (b - a) / N

# 定义f(x)函数


def f(x): return np.sin(np.pi * x)

# 定义基函数和它们的导数


def phi(i, x): return np.where((x >= (i-1)*h) & (x <= i*h), (x - (i-1)
                                                             * h) / h, np.where((x >= i*h) & (x <= (i+1)*h), ((i+1)*h - x) / h, 0))


def dphi(i, x): return np.where((x >= (i-1)*h) & (x <= i*h),
                                1 / h, np.where((x >= i*h) & (x <= (i+1)*h), -1 / h, 0))


# 定义最小二乘有限元法的矩阵和向量
A = diags([1/h, -2/h, 1/h], [-1, 0, 1], shape=(N+1, N+1)).toarray()
A[0, 0] = A[-1, -1] = 1
A[0, 1] = A[-1, -2] = 0
b = np.array([f((i+0.5)*h) for i in range(N+1)])

# 求解线性系统
u = spsolve(A, b)

# 计算解的误差
error = np.max(np.abs(u - np.sin(np.pi * np.linspace(a, b, N+1))))

print('最大误差:', error)

In [ ]:
# 文心一言
import numpy as np

# 定义节点和权值
x = np.linspace(0, 1, 100)
w = np.ones_like(x)

# 定义函数f(x)
f = np.sin(np.pi*x)

# 定义矩阵A
A = np.zeros((len(x), len(x)))
for i in range(len(x)):
    for j in range(i+1):
        A[i, j] = w[j] * (x[i] - x[j])

# 定义初值
u0 = np.zeros_like(x)

# 定义容差
tol = 1e-6

# 使用Gauss-Seidel迭代法求解最小二乘问题
u = u0
for k in range(1000):
    r = f - A @ u
    u_next = (A.T @ r + u) / A
    if np.linalg.norm(u_next - u) < tol:
        break
    u = u_next

# 输出结果
print("解：", u)